In [ ]:
# Newark Arrivals scrape proof of concept - This is isolated and not needed for core edct functionality.

from core.api.newark_departures import Newark_departures_scrape
nds = Newark_departures_scrape()
# soups = nds.soup_scrape_UA_arrivals()

# r=nds.extract_flight_id_and_link(soups=soups)


In [ ]:
# Part 1: Proof of concept - Mongo returns flights for particular arrival given datetime range - Mind the datetime format.
# This is supposed to leaad to the EDCT scrape proof of concept.
from config.database import collection_flights
def find_matching_flights(db_collection,
                            flight_number_list: list=None,
                            arrival_airport_icao='KMDW',
                            start_time='2025-12-01T12:00:00Z',
                            end_time='2025-12-02T03:00:00Z'
                            ):
    
    # This was used to pick out a list of flight numbers from the db.
    # flight_number_regex = '^|'.join(flight_number_list)

    pipeline = [
        # Match flights going to KEWR within our time window
        {
            '$match': {
                # 'flightID': {'$regex': flight_number_regex},
                'versions.arrival': arrival_airport_icao,
                'versions.estimatedDepartureTime': {
                    '$gte': start_time,
                    '$lt': end_time
                }
            }
        },
        # Unwind the versions array to process each version
        {'$unwind': '$versions'},
        # Filter versions to only KEWR arrivals in our time window
        {
            '$match': {
                'versions.arrival': arrival_airport_icao,
                'versions.estimatedDepartureTime': {
                    '$gte': start_time,
                    '$lt': end_time
                }
            }
        },
        # Group by flightID and take the first version (or sort first if you want most recent)
        {
            '$group': {
                '_id': '$flightID',
                'registration': {'$first': '$versions.registration'},
                'departure': {'$first': '$versions.departure'},
                'arrival': {'$first': '$versions.arrival'},
                'estimatedDepartureTime': {'$first': '$versions.estimatedDepartureTime'}
            }
        },
        # Project to clean up the output format
        {
            '$project': {
                '_id': 0,
                'flightID': '$_id',
                'registration': 1,
                'departure': 1,
                'arrival': 1,
                'estimatedDepartureTime': 1
            }
        }
    ]
    
    return list(db_collection.aggregate(pipeline))

flight_numbers = None
# matching_flights = find_matching_flights(collection_flights, flight_numbers)
matching_flights = find_matching_flights(
                        collection_flights, flight_numbers,
                        arrival_airport_icao='KPBI',
                        # PBI MCI DFW
                        # start_time='2025-08-20T13:00:00Z',
                        # end_time='2025-08-20T23:00:00Z'
                    )
len(matching_flights), matching_flights


In [ ]:
# Part 2: Proof of concept for EDCT scrape in bulk given mongo db collection returns in a particular format.

from core.EDCT_Lookup import EDCT_LookUp


edct = EDCT_LookUp()
flightID_data= {}
registration_data = {}
unsuccessful_flights = {}

for i in matching_flights:
    # print(i)
    # call_sign = i['registration'] if i['registration'] else i['flightID']
    call_sign = i.get('flightID')
    origin = i['departure'][1:]
    destination = i['arrival'][1:]
    results = edct.extract_edct(call_sign=call_sign, origin=origin, destination=destination)
    if results:
        flightID_data[i['flightID']] = {
            'registration': i.get('registration'),
            'origin': origin,
            'destination': destination,
            'estimatedDepartureTime': i['estimatedDepartureTime'],
            'results': results
        }
    else:
        # Trying when flightID shows none.
        registration = i.get('registration')
        if registration:
            call_sign = registration
            results = edct.extract_edct(call_sign=call_sign, origin=origin, destination=destination)
            if results:
                registration_data[i.get('registration')] = {
                    'origin': origin,
                    'destination': destination,
                    'estimatedDepartureTime': i['estimatedDepartureTime'],
                    'results': results
                }
            else:
                unsuccessful_flights[i['flightID']] = {
                    'registration': i.get('registration'),
                    'origin': origin,
                    'destination': destination,
                    'estimatedDepartureTime': i['estimatedDepartureTime'],
                    'results': results
                }

In [ ]:
[i for i in flightID_data.keys() if i[0]!= 'N']

In [ ]:
flightID_data
f = set()
for a,b in flightID_data.items():
    f.add(a[:3])
flightID_data


In [ ]:
# TODO test: Schedule tests duing ground stop using this script to get arrival airport and its flight numebrs
        # Use it to fetch edct data and asert using test on the frontend.
# flightID_data
# registration_data

# unsuccessful_flights airline codes and flights
f = set()
for a,b in unsuccessful_flights.items():
    f.add(a[:3])
f,unsuccessful_flights




In [ ]:
list(collection_flights.find({"flightID": "AAL1451"}))

In [ ]:
edct = EDCT_LookUp()
results = edct.extract_edct(call_sign="GJS4509", origin="ORF", destination="EWR")
results
